## Titel: Import data
## Author: Achraf Aboukinana

## Beschreibung des Codes

Dieser Code dient der Analyse, Bereinigung und Kombination von Datensätzen, um eine zentrale und bereinigte Datenbasis zu erstellen. Die Hauptaufgaben sind wie folgt:

---

## **1. Datenimport**
- Die drei Datensätze `kiwo.csv`, `umsatzdaten_gekuerzt.csv` und `wetterdaten.csv` werden eingelesen.
- Die Spalte `Datum` wird als Datumsformat (`datetime`) geparst.

---

## **2. Datenexploration**
Eine Funktion `explore_data` analysiert jeden DataFrame:
- Form des DataFrames (Zeilen und Spalten).
- Vorschau der ersten fünf Zeilen.
- Spaltennamen, Datentypen und fehlende Werte pro Spalte.
- Beschreibende Statistiken.
- Anzahl der Duplikate.
- Zeitraum der Daten (falls `Datum`-Spalte vorhanden).

Diese Funktion wird auf alle Datensätze angewendet.

---

## **3. Zusammenführung der Datensätze**
- **Erster Merge**: `umsatzdaten_gekuerzt_data` und `wetter_data` werden anhand der Spalte `Datum` zusammengeführt.
- **Zweiter Merge**: Die zusammengeführten Daten werden mit `kiwo_data` kombiniert, wodurch eine neue Spalte `KielerWoche` entsteht:
  - Fehlende Werte in `KielerWoche` (für nicht-Kieler-Woche-Tage) werden mit `0` gefüllt.

---

## **4. Analyse fehlender Werte**
- Anzahl der fehlenden Werte (`NaN`) pro Spalte wird berechnet.
- Zeilen mit fehlenden Werten werden entfernt.

---

## **5. Speichern der Ergebnisse**
- Der kombinierte Datensatz wird in zwei Versionen gespeichert:
  - **Mit NaN-Werten**: `final_data_withNaN.csv`.
  - **Ohne NaN-Werte**: `final_data.csv`.

---

## **6. Vergleich der Zeilenanzahl**
- Die Anzahl der Zeilen vor und nach der Bereinigung wird verglichen, um den Verlust durch das Entfernen von `NaN`-Werten zu zeigen.

---

## **Zweck des Codes**
1. **Datenbereinigung**: Sicherstellung von konsistenten und vollständigen Daten.
2. **Kombination der Datensätze**: Erstellung einer zentralen, einheitlichen Datenbasis.
3. **Markierung besonderer Tage**: Identifikation der Kieler Woche-Tage (`KielerWoche`).
4. **Analyse-Ready**: Der bereinigte Datensatz dient als Grundlage für Analysen oder Modelle.


In [113]:
import pandas as pd
import holidays
# Import Data
kiwo_data = pd.read_csv('kiwo.csv', parse_dates=['Datum'])
umsatzdaten_gekuerzt_data = pd.read_csv('umsatzdaten_gekuerzt.csv', parse_dates=['Datum'])
wetter_data = pd.read_csv('wetterdaten.csv', parse_dates=['Datum'])


In [114]:


# Funktion zur Datenexploration
def explore_data(df, name):
    print(f"\n{'='*50}\nExploration für {name}\n{'='*50}")
    
    # Überblick
    print(f"Form: {df.shape[0]} Zeilen, {df.shape[1]} Spalten")
    print("Erste 5 Zeilen:")
    print(df.head())
    
    # Spaltennamen und Typen
    print("\nSpalten und Datentypen:")
    print(df.dtypes)
    
    # Fehlende Werte
    print("\nFehlende Werte pro Spalte:")
    print(df.isna().sum())
    
    # Beschreibende Statistik
    print("\nBeschreibende Statistik:")
    print(df.describe(include='all'))
    
    # Duplikate
    duplicate_count = df.duplicated().sum()
    print(f"\nAnzahl der Duplikate: {duplicate_count}")

    # Wichtige Spalten
    if 'Datum' in df.columns:
        print("\nZeitraum der Daten:")
        print(f"Min Datum: {df['Datum'].min()}, Max Datum: {df['Datum'].max()}")
    else:
        print("\nKeine 'Datum'-Spalte in diesem DataFrame.")


In [ ]:

# Exploration der einzelnen Dateien
explore_data(kiwo_data, "DATA/Kiwo Data")
explore_data(umsatzdaten_gekuerzt_data, "DATA/Umsatzdaten Gekürzt")
explore_data(wetter_data, "DATA/Wetter Data")


In [ ]:


# Datei einlesen mit manuellem Trennzeichen
file_path = "DATA/inflation.csv"  # Pfad zur Datei
data = pd.read_csv(file_path, delimiter=',')



# Kategorisierung der Veränderung im Vorjahresmonat
def categorize_inflation(row):
    if row["Veränderung_Vormonat"] > 0:
        return "Positiv"
    elif row["Veränderung_Vormonat"] < 0:
        return "Negativ"
    else:
        return "Neutral"

# Neue Spalte für Inflation-Kategorisierung hinzufügen
data["Inflation_Kategorisierung_vormonat"] = data.apply(categorize_inflation, axis=1)

# Dummy-Encoding der Kategorisierungen
data = pd.get_dummies(data, columns=["Inflation_Kategorisierung_vormonat"], drop_first=True)

### nach paar änderungen habe ich den tabelle vorbereiten so dass wir es erstmal mergen und in unsere model benutzen 
# In eine neue CSV-Datei speichern
data.to_csv("DATA/inflation.csv", index=False)

# Daten anzeigen
print(data.head())



In [ ]:
import pandas as pd
import holidays

# Merge Umsatzdaten mit Wetterdaten auf 'Datum'
merged_data = pd.merge(umsatzdaten_gekuerzt_data, wetter_data, on='Datum', how='inner')

# Füge KielerWoche-Information hinzu und ersetze fehlende Werte durch 0
final_data = pd.merge(merged_data, kiwo_data[['Datum', 'KielerWoche']], on='Datum', how='left')
final_data['KielerWoche'] = final_data['KielerWoche'].fillna(0).astype(int)


In [ ]:

# Anzahl der NaN-Werte überprüfen
nan_counts = final_data.isna().sum()
print("Anzahl der NaN-Werte pro Spalte:")
print(nan_counts)

# Entferne Zeilen mit NaN-Werten
clean_data = final_data.dropna()


In [ ]:

# **Zeitabhängige Variablen hinzufügen**
clean_data['Wochentag'] = clean_data['Datum'].dt.dayofweek  # Montag=0, Sonntag=6
clean_data['Monat'] = clean_data['Datum'].dt.month
clean_data['Jahreszeit'] = clean_data['Datum'].dt.month % 12 // 3 + 1  # Frühling=1, Sommer=2, Herbst=3, Winter=4


In [129]:
clean_data= pd.read_csv('final_data_c.csv')
clean_data.head()


,Datum,Warengruppe,Umsatz,Bewoelkung,Temperatur,Wettercode,KielerWoche,Wochentag,Monat,Jahreszeit_1,...,Wind_Zu windig,Jahr,Verbraucherpreisindex,Veränderung_Vorjahresmonat,Veränderung_Vormonat,Inflation_Kategorisierung,Inflation_Kategorisierung_Neutral,Inflation_Kategorisierung_Positiv,Inflation_Kategorisierung_vormonat_Neutral,Inflation_Kategorisierung_vormonat_Positiv
0,2013-07-01,1,148.828353,6.0,17.8375,20.0,0,0,7,False,...,False,2013,93.5,2.0,0.4,Positiv,False,True,False,True
1,2013-07-03,1,111.885594,7.0,21.0750,61.0,0,2,7,False,...,False,2013,93.5,2.0,0.4,Positiv,False,True,False,True
2,2013-07-04,1,168.864941,7.0,18.8500,20.0,0,3,7,False,...,False,2013,93.5,2.0,0.4,Positiv,False,True,False,True
3,2013-07-08,1,135.500244,0.0,22.7250,5.0,0,0,7,False,...,False,2013,93.5,2.0,0.4,Positiv,False,True,False,True
4,2013-07-10,1,135.132314,2.0,19.6875,5.0,0,2,7,False,...,False,2013,93.5,2.0,0.4,Positiv,False,True,False,True


In [130]:



# Jahreszeit bestimmen
def get_season(month):
    if month in [12, 1, 2]:
        return "Winter"
    elif month in [3, 4, 5]:
        return "Frühling"
    elif month in [6, 7, 8]:
        return "Sommer"
    elif month in [9, 10, 11]:
        return "Herbst"

clean_data['Jahreszeit'] = clean_data['Monat'].apply(get_season)

# Gefühl abhängig von Temperatur und Jahreszeit
def temperature_feeling(temp, season):
    if season == "Winter":
        if temp <= 0:
            return "Kalt"
        elif temp <= 10:
            return "Mild"
        else:
            return "Warm"
    elif season == "Frühling":
        if temp <= 10:
            return "Kalt"
        elif temp <= 20:
            return "Mild"
        else:
            return "Warm"
    elif season == "Sommer":
        if temp <= 15:
            return "Kalt"
        elif temp <= 25:
            return "Mild"
        else:
            return "Warm"
    elif season == "Herbst":
        if temp <= 10:
            return "Kalt"
        elif temp <= 20:
            return "Mild"
        else:
            return "Warm"

clean_data['Gefühl'] = clean_data.apply(lambda x: temperature_feeling(x['Temperatur'], x['Jahreszeit']), axis=1)

# Dummy-Encoding
gefühl_dummies = pd.get_dummies(clean_data['Gefühl'],prefix='gefühl')

# Dummy-Variablen anhängen und Originalspalten löschen
clean_data = pd.concat([clean_data,gefühl_dummies], axis=1)
#gefühl_dummies.drop(columns=['Gefühl'], inplace=True)

clean_data.to_csv('final_data_x.csv',index=False)

In [ ]:

# **Dummy-Encoding für Jahreszeit und Wetterkategorie**
jahreszeit_dummies = pd.get_dummies(clean_data['Jahreszeit'], prefix="Jahreszeit")
wetter_dummies = pd.get_dummies(clean_data['Wetterkategorie'], prefix="Wetter")

# Dummy-Variablen anhängen und Originalspalten löschen
clean_data = pd.concat([clean_data, jahreszeit_dummies, wetter_dummies], axis=1)
clean_data.drop(columns=['Jahreszeit', 'Wetterkategorie'], inplace=True)


In [ ]:

# **Feiertage hinzufügen**
def get_feiertage(year):
    de_holidays = holidays.Germany(years=year)
    return list(de_holidays.keys())

# Feiertagsinformationen für alle Jahre im Datensatz
#feiertage_set = {holiday for year in clean_data['Datum'].dt.year.unique() for holiday in get_feiertage(year)}

# Feiertagsspalte erstellen
#clean_data['Feiertag'] = clean_data['Datum'].apply(lambda x: 1 if x.date() in feiertage_set else 0)

# **Besondere Events (KielLauf, Kieler Triathlon) hinzufügen**
kieler_triathlon_daten = pd.to_datetime(['2013-08-04', '2014-08-03', '2015-08-02', '2016-08-07', '2017-08-06', 
                                         '2018-08-05', '2019-08-04'])
kiellauf_daten = pd.to_datetime(["2013-09-08", "2014-09-14", "2015-09-13", "2016-09-11", 
                                 "2017-09-10", "2018-09-09", "2019-09-08"])
important_games_germany = pd.to_datetime([
    # FIFA World Cup 2014
    "2014-07-08",  # Germany 7–1 Brazil (Semifinal)
    "2014-07-13",  # Germany 1–0 Argentina (Final)

    # UEFA Euro 2016
    "2016-07-02",  # Germany 1–1 Italy (6–5 on penalties, Quarterfinal)
    "2016-07-07",  # Germany 0–2 France (Semifinal)

    # FIFA Confederations Cup 2017
    "2017-06-19",  # Germany 3–2 Australia
    "2017-06-22",  # Germany 1–1 Chile
    "2017-06-25",  # Germany 3–1 Cameroon
    "2017-06-29",  # Germany 4–1 Mexico (Semifinal)
    "2017-07-02",  # Germany 1–0 Chile (Final)

    # FIFA World Cup 2018
    "2018-06-17",  # Germany 0–1 Mexico (Group stage)
    "2018-06-27",  # Germany 0–2 South Korea (Group stage, eliminated)

    # Olympic Games 2016 (Men's Football)
    "2016-08-20"   # Germany 1–1 Brazil (4–5 on penalties, Final)
])

clean_data['KielLauf'] = clean_data['Datum'].apply(lambda x: 1 if x.date() in kiellauf_daten.date else 0)
clean_data['Kieler_Triathlon'] = clean_data['Datum'].apply(lambda x: 1 if x.date() in kieler_triathlon_daten.date else 0)
clean_data['Fußball'] = clean_data['Datum'].apply(lambda x: 1 if x.date() in important_games_germany.date else 0)


In [ ]:

# **PaycheckEffect (Gehaltszahlungen)**
clean_data['PaycheckEffect'] = clean_data['Datum'].apply(
    lambda x: 1 if x.day >= 27 or x.day <= 5 else 0
)

# Inflation sensitivity mapping based on WarenGruppe
sensitivity_mapping = {
    1: "High",   # Brot
    2: "High",   # Brötchen
    3: "Moderate",  # Croissant
    4: "Moderate",  # Konditorei
    5: "Moderate",  # Kuchen
    6: "High"    # Saisonbrot
}

# Map the sensitivity to the WarenGruppe
clean_data["InflationSensitivity"] = clean_data["Warengruppe"].map(sensitivity_mapping)

# Perform dummy encoding for sensitivity levels, excluding 'Low'
dummy_encoded = pd.get_dummies(clean_data["InflationSensitivity"], prefix="Sensitivity").drop(columns=["Sensitivity_Low"], errors="ignore")

# Concatenate the dummy columns with the original DataFrame
clean_data = pd.concat([clean_data, dummy_encoded], axis=1)


In [ ]:


# **Windkategorie basierend auf Windgeschwindigkeit**
def wind_kategorie(wind):
    if wind < 10:
        return "Nicht windig"
    elif wind <= 20:
        return "Windig"
    else:
        return "Sehr windig" #TODO zu => sehr

clean_data['Windkategorie'] = clean_data['Windgeschwindigkeit'].apply(wind_kategorie)

# Dummy-Variablen für Windkategorien erstellen
wind_dummies = pd.get_dummies(clean_data['Windkategorie'], prefix="Wind")
clean_data = pd.concat([clean_data, wind_dummies], axis=1)

# Originalspalten löschen
clean_data.drop(columns=['Windkategorie', 'Windgeschwindigkeit'], inplace=True)


In [ ]:

# **Speichern der Ergebnisse**
final_data.to_csv('final_data_withNaN.csv', index=False)  # Mit NaN-Werten
clean_data.to_csv('final_data.csv', index=False)         # Ohne NaN-Werte

# Zeilenanzahl vor und nach dem Entfernen von NaN
difference = len(final_data) - len(clean_data)
print(f"Zeilen mit NaN entfernt: {difference}")


In [ ]:

# **Zusammenfassung**
clean_data.head()
clean_data.info()
clean_data.describe()


In [ ]:
# Extract year from the date in Table 2
clean_data['Jahr'] = pd.to_datetime(clean_data['Datum']).dt.year
clean_data = pd.merge(clean_data,data,on = ['Jahr', 'Monat'] )
clean_data.to_csv('final_data_c.csv', index=False)         # Ohne NaN-Werte
